In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [25]:
# df_train = utils.read_df_pkl('../input/train0*')
# df_train.set_index(key, inplace=True)
df_hist = pd.read_csv('../input/historical_transactions.csv')
# df_new = pd.read_csv('../input/new_merchant_transactions.csv')

df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
df_new['purchase_amount_new'] = np.round(df_new['purchase_amount'] / 0.00150265118 + 497.06, 2)

In [18]:
df_new['purchase_amount_integer'] = df_new.purchase_amount_new.apply(lambda x: x == np.round(x))
print(df_new.groupby('purchase_amount_integer')['card_id'].count())

purchase_amount_integer
False    1122934
True      840097
Name: card_id, dtype: int64


In [22]:
df_new.groupby('purchase_amount_new')['card_id'].count().reset_index(name='count').sort_values('count',ascending=False).head()

,purchase_amount_new,count
4984,50.0,48336
1984,20.0,40550
2984,30.0,36755
9984,100.0,29707
984,10.0,26914


In [24]:
df_train['target_raw'] = 2**df_train['target']
df_train['target_raw'].head()

card_id
C_ID_92a2005557    0.566331
C_ID_3d0044924f    1.313042
C_ID_d639edf6cd    1.611111
C_ID_186d6a6901    1.103813
C_ID_cdbd2c0db2    0.895181
Name: target_raw, dtype: float64

In [36]:
merchant_counts = df_hist.groupby(['card_id'])['merchant_id'].nunique().reset_index(name = 'merchant_n')
one_merchant = merchant_counts[merchant_counts['merchant_n']==1].reset_index(drop=True)
dat = df_hist.loc[df_hist['card_id'].isin(one_merchant['card_id'])]
dat = dat.loc[~dat['card_id'].isin(df_new['card_id'])]
dat = dat.loc[dat.merchant_id=='M_ID_fc7d7969c3'].reset_index(drop=True)

In [30]:
df_hist[df_hist.card_id=='C_ID_b3c7ff9e19'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
27771438,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-13,-0.701979,2017-01-03 15:28:20,NaN,-1,8,29.9
27771445,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-12,-0.701979,2017-02-03 14:57:17,NaN,-1,8,29.9
27771441,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-11,-0.701979,2017-03-03 18:27:05,NaN,-1,8,29.9
27771447,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-10,-0.701979,2017-04-03 13:05:07,NaN,-1,8,29.9
27771449,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-9,-0.701979,2017-05-04 11:08:14,NaN,-1,8,29.9
27771437,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-8,-0.701979,2017-06-03 13:57:01,NaN,-1,8,29.9
27771444,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-7,-0.701979,2017-07-03 15:01:43,NaN,-1,8,29.9
27771440,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-6,-0.689957,2017-08-03 16:22:15,NaN,-1,8,37.9
27771446,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.689957,2017-09-03 15:59:53,NaN,-1,8,37.9
27771448,Y,C_ID_b3c7ff9e19,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.689957,2017-11-03 17:37:09,NaN,-1,8,37.9


In [31]:
df_hist.loc[df_hist.merchant_id=='M_ID_fc7d7969c3'].groupby('purchase_amount_new')['card_id'].count()

purchase_amount_new
1.00     45054
9.50         2
13.95        1
19.90    19709
22.90    31680
27.90    33017
29.90    24097
37.90    23480
Name: card_id, dtype: int64

In [37]:
dat = dat.merge(df_train, on = 'card_id')

In [34]:
dat.groupby('target_raw')['card_id'].count().reset_index(name='n')

,target_raw,n
0,1.000000e-10,23
1,2.180793e-02,13
2,5.000000e-01,32
3,6.666667e-01,2
4,7.132616e-01,6
5,7.991968e-01,5
6,9.998681e-01,3
7,1.000000e+00,2294
8,1.000132e+00,17
9,1.000179e+00,15


In [ ]:
dat.loc[dat['target_raw']==1,['card_id','purchase_date','purchase_amount_new','target_raw']].sort_values(['card_id','purchase_date'])

In [46]:
df_hist.loc[df_hist[key]=='C_ID_f840a74f9d'].sort_values(by='purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
28992117,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-7,-0.712497,2017-01-08 07:12:32,NaN,-1,8,22.9
28992114,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-6,-0.712497,2017-02-08 07:16:33,NaN,-1,8,22.9
28992116,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.712497,2017-03-08 09:21:24,NaN,-1,8,22.9
28992111,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.712497,2017-05-09 02:57:11,NaN,-1,8,22.9
28992113,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-2,-0.712497,2017-06-08 05:22:21,NaN,-1,8,22.9
28992115,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.712497,2017-07-08 05:15:23,NaN,-1,8,22.9
28992112,Y,C_ID_f840a74f9d,-1,Y,1,B,661,M_ID_fc7d7969c3,0,-0.704984,2017-08-08 05:11:41,NaN,-1,8,27.9
